In [1]:
import numpy as np
import enviroment_no_visual as enviroment

In [2]:
def exploration_policy():
    return np.random.randint(3)

In [3]:
def binary_vector_to_decimal(vector):    
    # Converti il vettore binario in una stringa binaria
    binary_string = ''.join(map(str, vector))
    decimal_value = int(binary_string, 2)
    return decimal_value

In [4]:
import numpy as np

# Crea un array con tutte le combinazioni di 0 e 1 di lunghezza n
def generate_boolean_vectors(n):
    return np.array(np.meshgrid(*[np.arange(2)] * n)).T.reshape(-1, n)

def initialize_QValues():
    Q_values = np.zeros((2048, 3))
    all_states = generate_boolean_vectors(11)
    for s in all_states:
        if (s[3] + s[4] + s[5] + s[6]) != 1: # controlla che esista una sola direzione
            impossible_row = binary_vector_to_decimal(s)
            Q_values[impossible_row, :] = -np.inf
        if (s[7] + s[8]) > 1 or (s[9] + s[10]) > 1 or ((s[7] + s[8] + s[9] + s[10]) == 0): # controlla che il frutto non sia a destra e a sinistra o sopra e sotto
            impossible_row = binary_vector_to_decimal(s)
            Q_values[impossible_row, :] = -np.inf
    return Q_values

In [5]:
def step(env, action):
    final_move = [0,0,0]
    final_move[action] = 1
    new_state, reward, game_over, _ = env.play_step_QL(final_move)
    return new_state, reward, game_over

In [6]:
alpha0 = 0.05  # initial learning rate
decay = 0.005  # learning rate decay
gamma = 0.90  # discount factor

env = enviroment.SnakeGameAI()
env.reset()
Q_values = initialize_QValues()

for iteration in range(1_000_000):
    action = exploration_policy()
    next_state, reward, game_over = step(env, action)
    row_next_state = binary_vector_to_decimal(next_state)
    next_value = Q_values[row_next_state].max()  # greedy policy at the next step
    alpha = alpha0 / (1 + iteration * decay)
    Q_values[row_next_state, action] *= 1 - alpha
    Q_values[row_next_state, action] += alpha * (reward + gamma * next_value)
    if game_over:
        env.reset()


In [7]:
np.save('Q-Value.npy', Q_values)

Calcolati i Q-value effettua la scelta della prossima mossa in base all'azione con maggiore Q-value

In [8]:
#Q_values = np.load('Q-Value.npy')
#env = enviroment.SnakeGameAI()
env.reset()
game_over = False
step=0
while not game_over:
    state= env.get_state()
    row_Q_value = binary_vector_to_decimal(state)
    action = np.argmax(Q_values[row_Q_value])
    final_move = [0,0,0]
    final_move[action] = 1
    new_state, reward, game_over, score = env.play_step_QL(final_move)

    step+=1
    
print(f"Score: {score}\nNumber of steps: {step}", end="")

Score: 0
Number of steps: 5

In [9]:
s1= [0,0,1,
 0,0,1,0,
 0,0,1,0]
print(Q_values[binary_vector_to_decimal(s1)])
print(Q_values[binary_vector_to_decimal(s1)].max())

[ 2.00212101e-06  1.63226371e-03 -2.87873048e-01]
0.0016322637145301543


In [18]:
import enviroment as enviroment_visual
env_visual = enviroment_visual.SnakeGameAI()
env_visual.reset()
game_over = False
step=0
while not game_over:
    state= env_visual.get_state()
    row_Q_value = binary_vector_to_decimal(state)
    action = np.argmax(Q_values[row_Q_value])
    final_move = [0,0,0]
    final_move[action] = 1
    new_state, reward, game_over, score = env_visual.play_step_QL(final_move)
    print(reward)
    step+=1
    
print(f"Score: {score}\nNumber of steps: {step}", end="")

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
-10
Score: 0
Number of steps: 19

: 